# Converting large json files to a SQL database

In [ ]:
import json
import sqlite3
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from io import StringIO

Dataset contains 71,826,552 comments

In [ ]:
json_database = create_engine('sqlite:///json_database.db')

In [ ]:
batch_no=1

for chunk in pd.read_json("data/RC_2016-10_10M_20M_line_delim.json",chunksize=10000,lines=True):
    try: 
        chunk.to_sql('chunk_sql', json_database, if_exists='append')
    except sqlalchemy.exc.SQLAlchemyError as e: 
        print("\n  {}".format(e.orig))
    print('\rindex: {}'.format(batch_no), end='')
    batch_no+=1

In [ ]:
# df_new = pd.read_sql_query('SELECT * FROM chunk_sql', json_database)
# df_new

In [ ]:
def extract_comments(filepath, limit=None, links_only=False):
    comments = []
    with open(filepath) as fp:
        for idx, line in enumerate(fp):
            if (limit is not None) and (idx == limit):
                break
            comment = json.loads(line)
            if links_only:
                if 'https' in comment['body']:
                    comments.append(comment)
            else:
                comments.append(comment)
        return comments
    
    
def extract_subset(filepath, start=0, end=10):
    comments = []
    with open(filepath) as fp:
        for idx, line in enumerate(fp):
            if (idx >= start) and (idx < end):
                comment = json.loads(line)
                comments.append(comment)
            elif idx >= end:
                break                
        return comments

In [ ]:
# comments = extract_comments(filepath='data/RC_2016-10.json', limit=1000, links_only=False)
comments = extract_subset(filepath='data/RC_2016-10.json', start=10000000, end=20000000)

In [ ]:
df = pd.DataFrame(comments)
df.head()

In [ ]:
len(df)

In [ ]:
# df.to_pickle("RC_2016-10.pkl")
df.to_json("data/RC_2016-10_10M_20M_line_delim.json", orient='records', lines=True)